# 4D Neural Network Inversion Training
This notebook generates a model for the notebook in [`02 - 4D-Inversion-Field-Data`](02 - 4D-Inversion-Field-Data).

In [ ]:
import os

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np

from scipy.io import loadmat, savemat

from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2

In [ ]:
from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

## Data Loading
The data was provided in Matlab data format.
It is available as Pressure, Gas Saturation and Water Saturation maps together and to increase training data, also available as isolated effects. Additionally, pore volume maps turned out to be essential for a successful inversion.

Each map contains values for $\Delta P$, $\Delta S_g$, $\Delta S_w$ maps and simulated seismic difference maps for near, mid and far in the `dSNA_syn_` words.

In [ ]:
def data_gen_train_full():
    seed(42)
    set_random_seed(42)
    location = "data"
    filename = "Seis2PS_NN_training_input"
    suffixes = ["", "_Ponly", "_SGonly", "_SWonly"]
    #suffixes = [""]
    out = pd.DataFrame()
    for suff in suffixes:
        seis_ps = loadmat(os.path.join(location,filename+suff))
        headers = ["dP", "dSg", "dSw", "dSNA_syn_nr", "dSNA_syn_md", "dSNA_syn_fr"]
        print(suff)
        for q in range(len(seis_ps["dSg"][0])):
            tost = pd.DataFrame()
            for x in headers:
                tost[x] = seis_ps[x][0][q].ravel()
            if suff == "":
                pore = "Pore_volume"
                pore_data = seis_ps[pore].ravel()
                tost[pore] = pore_data
            else:
                tost[pore] = pore_data
            out = out.append(tost)
    out = out.dropna()
    out = out.loc[~(out[["dP", "dSg", "dSw"]]==0).all(axis=1)]
    out = out.sample(frac=1).reset_index(drop=True)
    
    y_train = out[["dP", "dSg", "dSw"]]
    X_train = out[["dSNA_syn_nr", "dSNA_syn_md", "dSNA_syn_fr", "Pore_volume"]]
    
    
    
    
    return X_train, y_train


In [ ]:
def data_gen_test(q):
    seed(42)
    set_random_seed(42)
    location = "data"
    filename = "Seis2PS_NN_training_input"
    seis_ps = loadmat(os.path.join(location,filename))
    headers = ["dP", "dSg", "dSw", "dSNA_syn_nr", "dSNA_syn_md", "dSNA_syn_fr"]
    tost = pd.DataFrame()
    for x in headers:
        tost[x] = seis_ps[x][0][q].ravel()
    
    pore = "Pore_volume"
    pore_data = seis_ps[pore].ravel()
    tost[pore] = pore_data
    
    out = tost.dropna()
    out = out.loc[~(out[["dP", "dSg", "dSw"]]==0).all(axis=1)]

    y_train = out[["dP", "dSg", "dSw"]]
    X_train = out[["dSNA_syn_nr", "dSNA_syn_md", "dSNA_syn_fr", "Pore_volume"]]
    
    return X_train, y_train


## Model Building
The initial architecture was generated from an encoder decoder architecture using `hyperas` to optimize width, depth and dropout rate for predicting one synthetic map from the other time steps.

In [ ]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, AlphaDropout, Dropout, Lambda, GaussianNoise, BatchNormalization, Concatenate
from keras import regularizers
from keras import optimizers 
from keras import callbacks

from keras_tqdm import TQDMNotebookCallback 

#from hyperopt import Trials, STATUS_OK, tpe
#from hyperas import optim
#from hyperas.distributions import choice, quniform, uniform, loguniform

from sklearn.metrics import r2_score
%autoreload 2

In [ ]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

def r_square_loss(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return 1 - ( 1 - SS_res/(SS_tot + K.epsilon()))

def huber_loss(y_true, y_pred, clip_delta=.35):
    error = y_true - y_pred
    cond  = tf.keras.backend.abs(error) < clip_delta
    squared_loss = 0.5 * tf.keras.backend.square(error)
    linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)
    
    return tf.where(cond, squared_loss, linear_loss)

In [ ]:
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def build_vae(near_off mid_off, far_off, noise=0.01, lr=1e-4):   
    seed(42)
    set_random_seed(42)
    
    alpha_dropout = 0.20
    encoding_dims = 256
    growth_factor = 1
    
    layers = 4
    
    
    mid_near_offset = mid-near
    far_near_offset = far-near
    far_mid_offset  = far-mid
    
    near = Input(shape=(1,), name="near_input")
    mid = Input(shape=(1,), name="mid_input")
    far = Input(shape=(1,), name="far_input")
    pore = Input(shape=(1,), name="pore_input")
    noisy_near = GaussianNoise(noise)(near)
    noisy_mid = GaussianNoise(noise)(mid)
    noisy_far = GaussianNoise(noise)(far)
    mid_near = Lambda(lambda inputs: ( inputs[0] - inputs[1] ) / ( mid_near_offset ))([noisy_mid, noisy_near])
    far_near = Lambda(lambda inputs: ( inputs[0] - inputs[1] ) / ( far_near_offset ))([noisy_far, noisy_near])
    far_mid = Lambda(lambda inputs: ( inputs[0] - inputs[1] ) / ( far_mid_offset ) )([noisy_far, noisy_mid])
    
    input_gradient = Concatenate()([noisy_near, noisy_mid, noisy_far, mid_near, far_near, far_mid, pore])
    
    encoded = Dense(encoding_dims*growth_factor*layers, activation="relu", name="encoder_0")(input_gradient)
    encoded = Dropout(alpha_dropout)(encoded)
    #encoded = BatchNormalization()(encoded)
    
    for q in range(layers):
        encoded = Dense(encoding_dims*growth_factor*(layers-q), activation="relu", name="encoder_"+str(q+1))(encoded)
        encoded = Dropout(alpha_dropout)(encoded)
        #encoded = BatchNormalization()(encoded)
    
    z_mean = Dense(encoding_dims, name='z_mean')(encoded)
    z_log_var = Dense(encoding_dims, name='z_log_var')(encoded)
    
    # use reparameterization trick to push the sampling out as input
    # note that "output_shape" isn't necessary with the TensorFlow backend
    deep_down = Lambda(sampling, name='z')([z_mean, z_log_var])
    
    decoded0 = Dense(encoding_dims*growth_factor, activation="relu", name="decoder_0")(deep_down)
                        
    for q in range(2,layers):
        #decoded0 = BatchNormalization()(decoded0)
        decoded0 = Dropout(alpha_dropout)(decoded0)
        decoded0 = Dense(encoding_dims*growth_factor*q, activation="relu", name="decoder_"+str(q-1))(decoded0)
    
    output0 = Dense(encoding_dims*growth_factor*layers, activation="linear")(decoded0)
    dP = Dense(1, activation="linear", name="dP")(output0)
    output1 = Dense(encoding_dims*growth_factor*layers, activation="linear")(decoded0)
    dSw = Dense(1, activation="linear", name="dSw")(output1)
    output2 = Dense(encoding_dims*growth_factor*layers, activation="linear")(decoded0)
    dSg = Dense(1, activation="linear", name="dSg")(output2)
    
    model = Model(inputs=[near, mid, far, pore], output=[dP,dSw,dSg])
        
    model.compile(loss="mse", optimizer=optimizers.Nadam(lr=lr), metrics=[r_square])
    
    model.summary()
    
    return model


In [ ]:
X_train, y_train = data_gen_train_full()

In [ ]:
model  = build_vae(10,20,30,noise=0.00,lr=5e-4)

In [ ]:
earlystop0 = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5,verbose=1,mode='auto', restore_best_weights=True)
earlystop1 = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=11,verbose=1,mode='auto', restore_best_weights=True)
earlystop2 = callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=51,verbose=1,mode='auto', restore_best_weights=False)
ir1 = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, verbose = 1, min_lr=0)
ir2 = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=10, verbose = 1, min_lr=0)

## Pre-Training without Noise
Clearly the network should converge on the model data. This 

In [ ]:
result = model.fit(
    [X_train["dSNA_syn_nr"],X_train["dSNA_syn_md"],X_train["dSNA_syn_fr"],X_train["Pore_volume"]],
    [y_train["dP"],y_train["dSw"],y_train["dSg"]],
    batch_size= 400,
    epochs=2000,
    verbose=0,
    shuffle=True,
    validation_split=0.01,
    callbacks = [TQDMNotebookCallback(leave_inner=True,leave_outer=True),earlystop1,ir1],
)

In [ ]:
model.save("pre.hd5")

## Fine-Tuning On Noisy Data
The final model has to expect noisy data to be able to transfer it to field data, we therefore rebuild the model and load the weights from the pre-trained network. This model will not converge particularly nor will it perform well on the synthetic data.

In [ ]:
model  = build_vae(noise=0.02,lr=1e-4)
model.load_weights("pre.hd5")
model.compile(loss="mse", optimizer=optimizers.Nadam(lr=1e-4), metrics=[r_square])
result = model.fit(
    [X_train["dSNA_syn_nr"],X_train["dSNA_syn_md"],X_train["dSNA_syn_fr"],X_train["Pore_volume"]],
    [y_train["dP"],y_train["dSw"],y_train["dSg"]],
    batch_size= 500,
    epochs=2000,
    verbose=0,
    shuffle=True,
    validation_split=0.01,
    callbacks = [TQDMNotebookCallback(leave_inner=True,leave_outer=True),earlystop2,ir2],
)
model.save("bbest.hd5")

In [ ]:
experiment = "publication"

In [ ]:
model = keras.models.load_model("best.hd5", custom_objects={'r_square': r_square, 'huber_loss': huber_loss})

# Test
Generate the test data and evaluate the model on this data. You may notice that the "test" data is contained within the train data, which would be relevant, if the actual evaluation was done on the synthetic data. During the build phase, this map would be excluded from the train data to get valid results. (This is important.)

In [ ]:
X_test, y_test = data_gen_test(3)

In [ ]:
print("Evalutation on unseen data:")
print(model.evaluate([X_test["dSNA_syn_nr"],X_test["dSNA_syn_md"],X_test["dSNA_syn_fr"],X_test["Pore_volume"]], [y_test["dP"],y_test["dSw"],y_test["dSg"]]))

In [ ]:
%%timeit
preddata  = model.predict([X_test["dSNA_syn_nr"],X_test["dSNA_syn_md"],X_test["dSNA_syn_fr"],X_test["Pore_volume"]])

# Plotting
The plotting rearranges the sample-wise prediction into the original map shape and deletes predictions, where data is not available.

`model_shape` saves a mask of the data, we apply this map to the predictions to get rid of predictions on samples that did not contain _actual_ data.

In [ ]:
location = "data"
filename = "Seis2PS_NN_training_input"
q=3
pore_volume = loadmat(os.path.join(location,filename))["Pore_volume"]
seis_ps = loadmat(os.path.join(location,filename))
ravel_data = seis_ps["dSNA_syn_nr"][0][q].ravel()
model_shape = ~np.isnan(ravel_data)

plt.figure(figsize=(15,20))
plt.subplot(2, 2, 1)
vmax=np.max(np.abs(seis_ps["dSNA_syn_nr"][0][q]))
plt.imshow(seis_ps["dSNA_syn_nr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Near")
plt.subplot(2, 2, 2)
vmax=np.max(np.abs(seis_ps["dSNA_syn_md"][0][q]))
plt.imshow(seis_ps["dSNA_syn_md"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Mid")
plt.subplot(2, 2, 3)
vmax=np.max(np.abs(seis_ps["dSNA_syn_fr"][0][q]))
plt.imshow(seis_ps["dSNA_syn_fr"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Far")
plt.subplot(2, 2, 4)
vmax=np.nanmax(seis_ps["Pore_volume"])
plt.imshow(seis_ps["Pore_volume"], cmap="seismic_r", vmin=-vmax, vmax=vmax,  aspect='auto')
plt.colorbar()
plt.title("Pore Volume")
plt.savefig("Seismic-input.png")
plt.show()

In [ ]:
data = np.full_like(seis_ps["dSg"][0][0], np.nan)
counter = 0
for i, m in enumerate(model_shape):
    if m:
        data[np.unravel_index(i, data.shape)] = preddata[0][counter]
        counter += 1
vmax = np.nanmax([np.abs(data),np.abs(seis_ps["dP"][0][q])])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,10))

im = axes[0].imshow(data, cmap="seismic", vmin=-vmax, vmax=vmax, aspect='equal')
axes[0].set_title("Neural Network dP "+experiment)

im = axes[1].imshow(seis_ps["dP"][0][q], cmap="seismic", vmin=-vmax, vmax=vmax, aspect='equal')
axes[1].set_title("Test Data dP")

fig.tight_layout()
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.2, 0.01, 0.6])
fig.colorbar(im, cax=cbar_ax)

blerg = "dP"
mat_dict = {blerg+"_nn_data": data, blerg: seis_ps[blerg][0][q]}
savemat('matlab/'+blerg+'.mat', mat_dict)

fig.savefig("Bestfit-dP-"+experiment+".png")
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data-seis_ps["dP"][0][q], cmap="seismic", vmin=-vmax, vmax=vmax)
plt.title("Misfit dP "+experiment)
plt.colorbar()
plt.tight_layout()
plt.savefig("Bestfit_diff-dP-"+experiment+".png")
plt.show()

In [ ]:
data = np.full_like(seis_ps["dSg"][0][0], np.nan)
counter = 0
for i, m in enumerate(model_shape):
    if m:
        data[np.unravel_index(i, data.shape)] = preddata[1][counter]
        counter += 1

vmax = np.nanmax([np.abs(data),np.abs(seis_ps["dSw"][0][q])])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,10))

im = axes[0].imshow(data, cmap="seismic_r", vmin=-vmax, vmax=vmax, aspect='equal')
axes[0].set_title("Neural Network dSw "+experiment)

im = axes[1].imshow(seis_ps["dSw"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax, aspect='equal')
axes[1].set_title("Test Data dSw")

fig.tight_layout()
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.2, 0.01, 0.6])
fig.colorbar(im, cax=cbar_ax)

blerg = "dSw"
mat_dict = {blerg+"_nn_data": data, blerg: seis_ps[blerg][0][q]}
savemat('matlab/'+blerg+'.mat', mat_dict)

fig.savefig("Bestfit-dSw-"+experiment+".png")
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data-seis_ps["dSw"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax)
plt.title("Misfit dSw "+experiment)
plt.colorbar()
plt.tight_layout()
plt.savefig("Bestfit_diff-dSw-"+experiment+".png")
plt.show()

In [ ]:
data = np.full_like(seis_ps["dSg"][0][0], np.nan)
counter = 0
for i, m in enumerate(model_shape):
    if m:
        data[np.unravel_index(i, data.shape)] = preddata[2][counter]
        counter += 1

vmax = np.nanmax([np.abs(data),np.abs(seis_ps["dSg"][0][q])])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,10))

im = axes[0].imshow(data, cmap="seismic_r",  vmin=-vmax, vmax=vmax, aspect='equal')
axes[0].set_title("Neural Network dSg "+experiment)

im = axes[1].imshow(seis_ps["dSg"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax, aspect='equal')
axes[1].set_title("Test Data dSg")

fig.tight_layout()
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.2, 0.01, 0.6])
fig.colorbar(im, cax=cbar_ax)

blerg = "dSg"
mat_dict = {blerg+"_nn_data": data, blerg: seis_ps[blerg][0][q]}
savemat('matlab/'+blerg+'.mat', mat_dict)

fig.savefig("Bestfit-dSg-"+experiment+".png")
fig.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data-seis_ps["dSg"][0][q], cmap="seismic_r", vmin=-vmax, vmax=vmax)
plt.title("Misfit dSg "+experiment)
plt.colorbar()
plt.tight_layout()
plt.savefig("Bestfit_diff-dSg-"+experiment+".png")
plt.show()